In [1]:
import os
import cv2 as cv
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import time

PARENT_DIR = "/Users/emmawaters/Desktop/Dance/Prod/"
NUM_PARTS = 15
NUM_CHUNKS = 1

In [2]:
# FOR COLOR CODING THE NOTEBOOK
from IPython.display import HTML, display

def set_background(color):    
    script = (
        "var cell = this.closest('.jp-CodeCell');"
        "var editor = cell.querySelector('.jp-Editor');"
        "editor.style.background='{}';"
        "this.parentNode.removeChild(this)"
    ).format(color)
    
    display(HTML('<img src onerror="{}" style="display:none">'.format(script)))

In [3]:
# read_video_to_movement returns the list of joint positions found using openCV
# Also returns a count of the number of joints not found, to be used to measure accuracy

def read_video_to_movement(vidpath):
    data = []
    net = cv.dnn.readNetFromTensorflow("graph_opt.pb")
    cap = cv.VideoCapture(vidpath)

    BODY_PARTS = { "Nose": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
                   "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
                   "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "Background": 14 }

    POSE_PAIRS = [ ["Neck", "RShoulder"], ["Neck", "LShoulder"], ["RShoulder", "RElbow"],
                   ["RElbow", "RWrist"], ["LShoulder", "LElbow"], ["LElbow", "LWrist"],
                   ["Neck", "RHip"], ["RHip", "RKnee"], ["RKnee", "RAnkle"], ["Neck", "LHip"],
                   ["LHip", "LKnee"], ["LKnee", "LAnkle"], ["Neck", "Nose"] ]

    error = 0
    while cv.waitKey(1) < 0:
        hasFrame, frame = cap.read()
        if not hasFrame:
            break

        #frame = cv.rotate(frame, cv.ROTATE_180) #ONLY needed if raw iPhone data
        frameWidth = frame.shape[1]
        frameHeight = frame.shape[0]
        net.setInput(cv.dnn.blobFromImage(frame, 1.0, (368, 368), (127.5, 127.5, 127.5), swapRB=True, crop=False))
        out = net.forward()
        out = out[:, :15, :, :]

        assert(len(BODY_PARTS) == out.shape[1])

        points = []
        for i in range(len(BODY_PARTS)):
            heatMap = out[0, i, :, :]
            _, conf, _, point = cv.minMaxLoc(heatMap)
            x = (frameWidth * point[0]) / out.shape[3]
            y = (frameHeight * point[1]) / out.shape[2]
            if conf > 0.1:
                points.append(np.array([x, y]))
            else:
                points.append(np.array([None,None]))
                error += 1

        data.append(points)
    return (data, error)

In [4]:
# COLLECT VIDEOS

song_list = ['Aurora', 'Aphex', 'Armatrading', 'Tnertle', 'Willow', 'Jiggle']
dancer_list = ['Dorissa', 'Dot', 'Falcon', 'Imogen', 'Lia', 'Sophia', 'Sunny']
vid_names = []
dir_list = os.listdir(PARENT_DIR)
for i in dir_list:
    if i.endswith(".mp4"):
        vid_names.append(i)
        
raw_name_dict = {'Aurora':[], 'Aphex':[], 'Armatrading':[], 'Tnertle':[], 'Willow':[], 'Jiggle':[]}
for vid in vid_names:
    name_split = vid.split('.')[0].split('_')
    raw_name_dict[name_split[1]].append(vid)

for i in raw_name_dict:
    raw_name_dict[i].sort()

In [5]:
# COLLECT RAW MOTION TRACKING DATA AND COLLECT ERRORS
FRAMES_BY_SONG = {}
raw_dance_data = {}
errors_by_dance = {'Aurora':[], 'Aphex':[], 'Armatrading':[], 'Tnertle':[], 'Willow':[], 'Jiggle':[]}

print(time.gmtime())

for song in song_list:
    print(song)
    num_frames = float('inf')
    song_raw_data = []
    for dancer in raw_name_dict[song]:
        print(dancer)
        (movement, error) = read_video_to_movement(os.path.join(PARENT_DIR, dancer))
        errors_by_dance[song].append(error/(len(movement)*NUM_PARTS))
        song_raw_data.append(movement)
        if len(movement) < num_frames:
            num_frames = len(movement)
    
    FRAMES_BY_SONG[song] = num_frames
    raw_dance_data[song] = song_raw_data  

print(time.gmtime())

time.struct_time(tm_year=2023, tm_mon=3, tm_mday=8, tm_hour=17, tm_min=46, tm_sec=0, tm_wday=2, tm_yday=67, tm_isdst=0)
Aurora
Dorissa_Aurora.mp4
Dot_Aurora.mp4
Falcon_Aurora.mp4
Imogen_Aurora.mp4


KeyboardInterrupt: 

In [ ]:
# COMPUTE CHARACTERISTIC VECTORS PER DANCE

def vectorize_movement(movement, num_chunks, NUM_PARTS, NUM_FRAMES):
    move_vec = []
    chunk_size = int(NUM_FRAMES/num_chunks)
    for n in range(num_chunks):
        for joint in range(NUM_PARTS):
            x_pos_disp = 0
            x_neg_disp = 0
            y_pos_disp = 0
            y_neg_disp = 0

            for i in range(chunk_size - 1):
                prev_frame = movement[n*chunk_size + i]
                frame = movement[n*chunk_size + i + 1]

                if prev_frame[joint].all() and frame[joint].all():
                    disp = prev_frame[joint] - frame[joint]

                    if disp[0] > 0:
                        x_pos_disp += disp[0]
                    else:
                        x_neg_disp += disp[0]
                    if disp[1] > 0:
                        y_pos_disp += disp[1]
                    else:
                        y_neg_disp += disp[1]
            move_vec += [x_pos_disp, x_neg_disp, y_pos_disp, y_neg_disp]
    return move_vec

characteristic_vectors = {}
for song in song_list:
    characteristic_vectors[song] = []
    for raw_motion in raw_dance_data[song]:
        move_vec = vectorize_movement(raw_motion, NUM_CHUNKS, NUM_PARTS, FRAMES_BY_SONG[song])
        characteristic_vectors[song].append(np.array(move_vec))

In [ ]:
# COMBINE VECTORS INTO A DATAFRAME
X_dict = {}
for song in song_list:
    X = np.array(characteristic_vectors[song])
    
    X_scaled = StandardScaler().fit_transform(X)
    X_dict[song] = X_scaled

In [ ]:
# PRODUCING PLOTS

def show_scree(song, X):
    pca_7comp = PCA(n_components=7)
    pca_7fit = pca_7comp.fit(X)

    plt.bar(
        range(1,len(pca_7comp.explained_variance_ratio_)+1),
        pca_7comp.explained_variance_ratio_
        )
    plt.xlabel('PCA Feature')
    plt.ylabel('Explained variance')
    plt.title('Feature Explained Variance ' + song)
    plt.show()
    print(pca_7comp.explained_variance_ratio_)

def show_pca_2plot(song, X):
    pca_2comp = PCA(n_components=2)
    pca_2fit = pca_2comp.fit_transform(X)
    
    pca_df = pd.DataFrame(data=pca_2fit, columns=['PC1', 'PC2'])
    pca_df['target'] = ['Dorissa', 'Dot', 'Falcon', 'Imogen', 'Lia', 'Sophia', 'Sunny']
    
    sns.set()
    sns.lmplot(
        x='PC1', 
        y='PC2', 
        data=pca_df, 
        hue='target', 
        fit_reg=False, 
        legend=True
        )

    plt.title(song)
    plt.show()
    
# GENERATE PCA TABLES & PLOTS
for song in song_list:
    show_scree(song, X_dict[song])
    show_pca_2plot(song, X_dict[song])
    

In [ ]:
print(errors_by_dance)

In [ ]:
# ERROR AVERAGING
set_background('red')

for song in song_list:
    sum_errors = 0
    for error in errors_by_dance[song]:
        sum_errors += error
    avg_error = sum_errors/7
    print(song + ": " + str(avg_error))

In [ ]:
# VALIDATION DATA
set_background('red')

for song in song_list:
    distances_by_dancer = {} # "name":(dis charactersitic, dist pca)
    for dancer in dancer_list:
        for comp_dancer in dancer_list:
            dist_char = 

In [ ]:
# DATA PER DANCER (OG IMPROV VIDS)
set_background('yellow')
char_vectors_by_dancer = [['Dorissa', []], ['Dot',[]], ['Falcon',[]], ['Imogen',[]], ['Lia',[]], ['Sophia',[]], ['Sunny',[]]]
for song in song_list:
    for i in range(7):
        
